In [1]:
import csv
import numpy as np
from collections import defaultdict
from matplotlib import pyplot
import pandas

In [2]:
BLANKS=["","RBNF1", "RBNF2", "RBNF", "0", "RBNF 1", "RBNF 2"]
#TOTAL= 5048
TOTAL=2734
#TOTAL=1351
#TOTAL=1201
#TOTAL=1168

In [3]:
def loadCsvData(fileName):
	matrix = []
	# open a file
	with open(fileName) as f:
		reader = csv.reader(f)
        # loop over each row in the file
		for row in reader:

			# cast each value to a float
			doubleRow = []
			for value in row:
				doubleRow.append(value.strip())

			# store the row into our matrix
			matrix.append(doubleRow)
	return matrix

In [4]:
data= loadCsvData("data/metadata.csv")

In [5]:
data=pandas.DataFrame(data[1:], columns=data[0])

In [6]:
interviewtxt =loadCsvData("data/interview text file to ids.csv")

In [7]:
interviewtxt=pandas.DataFrame(interviewtxt[1:], columns=interviewtxt[0])


In [8]:
text_to_ids=defaultdict(lambda: defaultdict(lambda:"0"))
for i in range(len(interviewtxt)):
    if interviewtxt["no_transcript"][i]=="TRUE":
        continue
    name=interviewtxt.at[i, interviewtxt.columns[0]]
    name=name[:name.rfind('.')]
    name=name.replace('\'','_')
    text_to_ids[name]["id"]=interviewtxt["interviewee_ids"][i].split(';')
    date=interviewtxt["date_of_first_interview"][i]
    text_to_ids[name]["interview year"]= date[date.rfind("/")+1:]

In [9]:
ids_to_years=defaultdict(lambda: defaultdict(lambda:"0"))
for i in range(len(interviewtxt)):
    date=interviewtxt["date_of_first_interview"][i]
    year=date[date.rfind('/')+1:]
    ids=interviewtxt["interviewee_ids"][i].split(';')
    no_transcript=interviewtxt["no_transcript"][i]
    for num in ids:
        ids_to_years[num]["year"]= str(year)
        if no_transcript=="TRUE" and ids_to_years[num]["included"]=="0":
            ids_to_years[num]["included"]=False
        else:
            ids_to_years[num]["included"]=True

In [10]:
database=defaultdict(lambda: defaultdict(lambda:0))
for i in range(len(data)):
    if ids_to_years[data.loc[i, "interviewee_id"]]["included"]==False:
        continue
    for atr in data.columns:
        if atr== "researcher_assumed_birth_decade":
            continue
        if atr== "researcher_assumed_race":
            continue
        value= data[atr][i]
        if value in BLANKS and atr=="\ufeffbirth_decade":
            value= data.loc[i, "researcher_assumed_birth_decade"]
        if value in BLANKS and atr=="identified_race":
            value= data.loc[i, "researcher_assumed_race"]
        if value in BLANKS and atr == "real_interviewee_birth_year":
            value= data.loc[i, "approximate_interviewee_birth_year"]
        database[data.loc[i, "interviewee_id"]][atr]=value
    database[data.loc[i, "interviewee_id"]]["interview_year"]=ids_to_years[data.loc[i, "interviewee_id"]]["year"]

In [21]:
def sort_by_birth_year(filenames):
    txt=""
    for i in filenames:
        myfile = open(i, encoding="latin1")
        read = myfile.read()
        myfile.close()
        txt+=read
    sectioned=[]
    string=""
    for i in txt:
        if i == '<':
            if len(string)>0:
                sectioned.append(string)
                string=""
        else:
            string+=i
    more_sectioned=[(i[i.rfind('\\\\')+2:i.rfind('>')], i[i.rfind('>'):]) for i in sectioned]
    more_sectioned.sort(key=lambda i: database[text_to_ids[i[0]]["id"][0]]["real_interviewee_birth_year"])
    return more_sectioned

In [17]:
Sexual_assault_second_hand= sort_by_birth_year(["data/Second_hand_intersect_sexual_assault.txt"])

In [19]:
f = open("Sexual_harassment_second_hand.txt", "a")
for i in Sexual_assault_second_hand:
    f.write(i[0])
    f.write(i[1][6:])
f.close()

In [22]:
all_sexual_assault=sort_by_birth_year(["data/Personal_experience_intersect_sexual_assault.txt", "data/Second_hand_intersect_sexual_assault.txt"])

In [23]:
f = open("Sexual_assault_first_and_second_hand.txt", "a")
for i in all_sexual_assault:
    f.write(i[0])
    f.write(i[1][6:])
f.close()

In [24]:
all_harassment=sort_by_birth_year(["data/all_sexual_harassment.txt"])

In [26]:
f = open("all_sexual_harassment.txt", "a")
for i in all_harassment:
    f.write(i[0])
    f.write(i[1][6:])
f.close()

In [27]:
first_and_second_hand_harassment=sort_by_birth_year(["data/SH_personal_exp.txt", "data/SH_second_hand.txt"])

In [28]:
f = open("first_and_second_hand_sexual_harassment.txt", "a")
for i in first_and_second_hand_harassment:
    f.write(i[0])
    f.write(i[1][6:])
f.close()